In [ ]:
# -*- coding: utf-8 -*-
"""
Author: Wang Jingyi, Wang Qi, Gu Yichao
Date: Sep 18, 2020
Purpose: For safe entry management
"""

import os
import pandas as pd
import datetime as dt
work_dir=os.getcwd()
sub_dir=f'{work_dir}\\INOUT'


def merge_csvs():
    # Combine in/out records in one data frame
    all_list=os.listdir(sub_dir)
    # Create seperate list for IN and OT file name
    list_in=[]
    list_ot=[]
    for item in all_list:
        if item[0]=='I':
            list_in.append(item)
        else:
            list_ot.append(item)
    
    # Load csv file data into IN and OT records
    in_records=[]
    for i in range(0,len(list_in)):
        path_in=f'{sub_dir}\\{list_in[i]}'
        each_in = pd.read_csv(path_in)
        in_records.append(each_in)   
    df_in = pd.concat(in_records,ignore_index=True)
        
    ot_records=[]
    for i in range(0,len(list_ot)):
        path_ot=f'{sub_dir}\\{list_ot[i]}'
        each_ot = pd.read_csv(path_ot)
        ot_records.append(each_ot)
    df_ot = pd.concat(ot_records,ignore_index=True)
    
    
    # Merge in/out records and modify the result.
    df_merge=pd.merge(df_in, df_ot,how='inner',on=['NRIC','Date'])
    # Change the header name
    df_merge.columns = ['Date', 'InTime','InGate','InPC','NRIC','ContactNo',
                    'OutTime','OutGate', 'OutPC']
    # Adjust  column order
    df_merge1 = df_merge.loc[:,['Date','InTime','InGate','InPC','ContactNo',
                    'OutGate','OutTime', 'OutPC','NRIC']]
    
    # # Calculate duration time
    TimeOutList = list(df_merge1['OutTime'])
    TimeInList = list(df_merge1['InTime'])
    StayMinsDurationList = []
    for i in range(0,len(TimeOutList)):
        TimeOut = dt.datetime.strptime(TimeOutList[i],'%H:%M')
        TimeIn = dt.datetime.strptime(TimeInList[i],'%H:%M')
        StayMinsDurationList.append((TimeOut-TimeIn).seconds//60)
    df_merge1['StayMinsDuration'] = StayMinsDurationList
    # # filter rows only when StayMinDuration is positive
    df_merge2=df_merge1[df_merge1['StayMinsDuration']>=0]
    # ##    For people that enter/exit more than once in the same day,
    # ##    we delete duplicates wit same Date, NRIC and Intime after sorting
    # ##    to make sure in/out records merge properly.
    
    df_merge_sort=df_merge2.sort_values(by=['Date','InTime','StayMinsDuration'],ascending=True)
    df_merge_final=df_merge_sort.drop_duplicates(subset=['Date','NRIC','InTime'],keep='first')
    
    # del df_merge_final['NRIC']
    # export the merge into csv
    path_merge=f'{work_dir}\\merged_output.csv'
    df_merge_final.to_csv(path_merge,index=False,mode='w')



if (__name__=="__main__"):
    merge_csvs()

In [ ]:
# -*- coding: utf-8 -*-
"""
Author: Wang Jingyi, Wang Qi, Gu Yichao
Date: Sep 18, 2020
Purpose: For safe entry management
"""

import datetime as dt
import csv
import os
import merge
import time
# Merge is the module we designed to merge csv files. 

work_dir=os.getcwd()
sub_dir=f'{work_dir}\\INOUT'

# Function for option C: configure PC number.
def acceptInteger1To99(question, error):
    while True:
        PCno = input(question)
        if (PCno.isnumeric() and 1 <= int(PCno) <= 99):
            break
        else:
            print(error)
    return PCno

# Function for option R: check if gateID is valid.
def is_valid_gateID():
    while True:
        gateID=input('Please enter a valid Door Gate ID(not more than 2 characters)\n')
        if (gateID.isalnum() and len(gateID)<=2):
            break
        else:
            pass
    return (gateID, True)
    

# Function for option R: check if PCno is valid.
def is_valid_PCno(PCno):
    try:
        1 <= int(PCno) <= 99
        return True
    except:
        return False

# Function for option R: write data to csv file
def check_and_add(path,header,data):
    try:
        f=open(path,'r')
        f.close()
        csv_write=open(path,'a+', newline='')
        writer = csv.writer(csv_write)
        writer.writerow(data)
    except:
        csv_write=open(path,'a+', newline='')
        writer = csv.writer(csv_write)
        writer.writerow(header)
        writer.writerow(data)

    csv_write.close()
    return None

# Function for option R: choose mode and capture data. 
def R_mode(gateID, PCno):
    while True:
        ## third input ID
        print('*****new record*****')
        ID_enter=input('Please scan your ID barcode: \n')
        ## forth choosing mode
        print(f'''
***** Choose your Mode *****
e: Entrance (IN)
x: Exit (OT)
Q: Quitting''')
        mode_cmd=input('Please enter your mode choice: \n')
        mode_cmd=mode_cmd.strip()
    
        # entrance
        if mode_cmd=='e':
            # enter the contact number
            contact_no=input('Please enter your contact number: \n')
            # generate different time strings
            now=dt.datetime.now()
            daynow1=dt.datetime.strftime(now,'%Y%m%d')
            daynow2=dt.datetime.strftime(now,'%Y-%m-%d')
            round_timenow=dt.datetime.strftime(now,'%H00')
            real_timenow=dt.datetime.strftime(now,'%H:%M')
          
            # parameters for export
            path_entrance=f'{sub_dir}//IN_{daynow1}_{gateID}_{PCno}_{round_timenow}.csv'
            e_header=['Date', 'TimeIn', 'GateIn','PCIn','NRIC','ContactNo']
            e_data = [daynow2,real_timenow,gateID,PCno,ID_enter,contact_no]
            
            # check if there is sub_dir, create here if not.
            if os.path.isdir(sub_dir)==False:
                os.mkdir(sub_dir)
            else: pass
            
            # write into csv files
            check_and_add(path_entrance,e_header,e_data)
        
        # exit
        elif mode_cmd=='x':
            # generate different time strings
            now=dt.datetime.now()
            daynow1=dt.datetime.strftime(now,'%Y%m%d')
            daynow2=dt.datetime.strftime(now,'%Y-%m-%d')
            round_timenow=dt.datetime.strftime(now,'%H00')
            real_timenow=dt.datetime.strftime(now,'%H:%M')
          
            # parameters for export
            path_exit=f'{sub_dir}//OT_{daynow1}_{gateID}_{PCno}_{round_timenow}.csv'
            x_header=['Date', 'TimeOut', 'GateOut','PCOut','NRIC']
            x_data = [daynow2,real_timenow,gateID,PCno,ID_enter]
            
            # check if there is sub_dir, create here if not.
            if os.path.isdir(sub_dir)==False:
                os.mkdir(sub_dir)
            else: pass
            
            # write into csv files
            check_and_add(path_exit,x_header,x_data)
        
        # quit  
        elif mode_cmd=='Q':
            break

        else: 
            print('Please enter a valid mode choice.')
            continue

# main function
def main():
    global PCno
    PCno = '' # Initial blank
    while True:
        print(f'''
***** Foot Print Contact Point *****
D: Set Door Gate
C: Configure PC number
R: Entrance & Exit Tracking
M: Merge Input/ Output Files
Q: Quit''')
        cmd=input('Please enter your command character: \n')
        cmd=cmd.strip() # Remove blank space.
        
        if cmd == 'D':
            global gateID
            gateID, gateID_is_valid = is_valid_gateID()
            continue
        
        if cmd == "C":
            PCno = acceptInteger1To99('Please enter PC Number (1 to 99)', 
                                      'Invalid entry, please enter any number from 1 to 99 only')
    
        elif cmd=='R':
            print('*** Welcome to the Entrance & Exit Tracking Session ***')
            ## first check gateID
            try:
                gateID_is_valid == True
                # Write gateID to txt
                gateID_path = 'ID-DoorGate.txt'
                with open(gateID_path, 'w') as f_gateID:                
                    f_gateID.write(gateID)
                
                # second check PCno. If it's invalid, set one here. 
                while is_valid_PCno(PCno) == False:
                    print('PC number is invalid!')
                    PCno = acceptInteger1To99('Please enter PC Number (1 to 99)', 
                                              'Invalid entry, please enter any number from 1 to 99 only')                   
                    if is_valid_PCno(PCno):
                        break
                    
                # Write PCno to txt
                PCno_path = 'ID-PCNumber.txt'
                with open(PCno_path, 'w') as f_PCno:
                    f_PCno.write(PCno)
            
                # choose mode and write entrance and exit details to CSVs
                R_mode(gateID, PCno)
    
            ## if gateID check fails, press any key to the menu
            except:
                input('Please enter a valid door gate before recording visits, press any key to return to the menu.\n')
        
        elif cmd=='M':
            timeStart=time.perf_counter() ##record start time
            merge.merge_csvs()
            timeEnd=time.perf_counter()
            timeDur=timeEnd-timeStart
            print(timeDur)
            print('*****All records are merged!!!*****')
        
        elif cmd=='Q':
            print('Bye!')
            break
        
        else:
            print('Invalid command character.')
            continue

if (__name__=='__main__'):
    main()
    
